In [ ]:
import os, json, random, cv2
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf, re, math
from tqdm import tqdm

from PIL import Image

# Intro

This notebook extends the work done by [ks2019](https://www.kaggle.com/ks2019) in the notebook [HappyWhale TFRecords](https://www.kaggle.com/ks2019/happywhale-tfrecords), to include:

* the [Yolov5 bounding box predictions](https://www.kaggle.com/awsaf49/happywhale-cropped-dataset-yolov5) created by [awsaf49](https://www.kaggle.com/awsaf49)
* the [Detic bounding box predictions](https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/305503) created by [phalanx](https://www.kaggle.com/phalanx).

In [ ]:
train_df = pd.read_csv('../input/happywhale-splits/skf_species_10folds.csv')
test_df = pd.read_csv('../input/happy-whale-and-dolphin/sample_submission.csv')
test_df['split'] = test_df.index%10

# Yolov5

Load the [Yolov5 bounding box predictions](https://www.kaggle.com/awsaf49/happywhale-cropped-dataset-yolov5) created by [awsaf49](https://www.kaggle.com/awsaf49).

In [ ]:
yolo5_train_df = pd.read_csv('../input/happywhale-cropped-dataset-yolov5/train.csv')
yolo5_test_df = pd.read_csv('../input/happywhale-cropped-dataset-yolov5/test.csv')

In [ ]:
yolo5_train_df.head()

In [ ]:
train_df['yolo5_bbox'] = yolo5_train_df.bbox.str[2:-2].str.replace(',', '')
test_df['yolo5_bbox'] = yolo5_test_df.bbox.str[2:-2].str.replace(',', '')

In [ ]:
train_df.head()

In [ ]:
def read_bbox(bbox):
    return np.array([int(i) for i in bbox.split()])

In [ ]:
def show_crop(row, crops=None, dataset='train'):
    crops = crops or {'yolov5', 'detic'}
    f, axarr = plt.subplots(1, len(crops) + 1, figsize=(18, 6))
    img = Image.open(f'../input/happy-whale-and-dolphin/{dataset}_images/{row.image}')
    
    arr_num = 0
    axarr[arr_num].imshow(img)

    if 'detic' in crops:
        detic_crop = img.crop(read_bbox(row.detic_bbox))
        arr_num += 1
        axarr[arr_num].set_title('Detic')
        axarr[arr_num].imshow(detic_crop)

    if 'yolov5' in crops:
        yolo5_crop = img.crop(read_bbox(row.yolo5_bbox))
        arr_num += 1
        axarr[arr_num].set_title('Yolov5')
        axarr[arr_num].imshow(yolo5_crop)

    plt.show()

In [ ]:
row = train_df.iloc[1610]
row

In [ ]:
show_crop(row, {'yolov5'})

# Detic

Load the [Detic bounding box predictions] created by [phalanx](https://www.kaggle.com/phalanx).

In [ ]:
detic_train_df = pd.read_csv('../input/whale2-cropped-dataset/train2.csv')
detic_test_df = pd.read_csv('../input/whale2-cropped-dataset/test2.csv')

In [ ]:
detic_train_df.head()

In [ ]:
detic_train_df.loc[detic_train_df.box.isna(), 'box'] = ''
detic_test_df.loc[detic_test_df.box.isna(), 'box'] = ''

In [ ]:
train_df['detic_bbox'] = detic_train_df.box
test_df['detic_bbox'] = detic_test_df.box

In [ ]:
row = train_df.iloc[1610]
row

In [ ]:
show_crop(row, {'detic'})

Any row missing bounding boxes, will use values -1, -1, -1, -1 as null value.

In [ ]:
train_df.loc[train_df.yolo5_bbox == '', 'yolo5_bbox'] = '-1 -1 -1 -1'
train_df.loc[train_df.detic_bbox == '', 'detic_bbox'] = '-1 -1 -1 -1'

test_df.loc[test_df.yolo5_bbox == '', 'yolo5_bbox'] = '-1 -1 -1 -1'
test_df.loc[test_df.detic_bbox == '', 'detic_bbox'] = '-1 -1 -1 -1'

# Setup Dataset

In [ ]:
%%time

### Create Kaggle Dataset if not exists 
DATASET_NAME = f'happywhale-tfrecords-bb'

!rm -rf /tmp/{DATASET_NAME}

os.makedirs(f'/tmp/{DATASET_NAME}', exist_ok=True)

with open('../input/kaggleapi/kaggle.json') as f:
    kaggle_creds = json.load(f)
    
os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

!kaggle datasets init -p /tmp/{DATASET_NAME}

with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json') as f:
    dataset_meta = json.load(f)

dataset_meta['id'] = f'lextoumbourou/{DATASET_NAME}'
dataset_meta['title'] = DATASET_NAME
with open(f'/tmp/{DATASET_NAME}/dataset-metadata.json', "w") as outfile:
    json.dump(dataset_meta, outfile)
print(dataset_meta)

!cp /tmp/{DATASET_NAME}/dataset-metadata.json /tmp/{DATASET_NAME}/meta.json
!ls /tmp/{DATASET_NAME}

#!kaggle datasets create -u -p /tmp/{DATASET_NAME} 

# Create TFRecords

In [ ]:
def get_fold(fold):
    val_df = train_df[train_df.fold==fold].reset_index(drop=True)
    val_df['order'] = val_df.index
    val_df['order'] = val_df.groupby('individual_id').order.rank()
    val_total_counts = val_df.individual_id.value_counts().to_dict()
    val_df['total_counts'] = val_df.individual_id.map(val_total_counts)
    val_df['order'] = val_df['order']/val_df['total_counts']
    val_df = val_df.sort_values('order',ascending=False).reset_index(drop=True)
    val_df = val_df[['image','species','individual_id','yolo5_bbox', 'detic_bbox']]
    return val_df

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bb_feature(bb):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=bb))

def serialize_example(image,image_name,target,species,yolov5_bb,detic_bb):
    feature = {
        'image': _bytes_feature(image),
        'image_name': _bytes_feature(image_name),
        'target': _int64_feature(target),
        'species': _int64_feature(species),
        'yolov5_box': _bb_feature(yolov5_bb),
        'detic_box': _bb_feature(detic_bb)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
def create_tf_records(fold  = 0, progress=False):
    df = get_fold(fold)
    tfr_filename = f'/tmp/{DATASET_NAME}/happywhale-2022-train-{fold}-{df.shape[0]}.tfrec'
    with tf.io.TFRecordWriter(tfr_filename) as writer:
        it = df.iterrows()
        if progress:
            it = tqdm(it, total=len(df))
        for i,row in it:
            image_id = row.image
            target = row.individual_id
            species = row.species
            image_path = f"../input/happy-whale-and-dolphin/train_images/{image_id}"
            image_encoded = tf.io.read_file(image_path)
            image_name = str.encode(image_id)
            yolov5_bb = list(read_bbox(row.yolo5_bbox))
            detic_bb = list(read_bbox(row.detic_bbox))
            example = serialize_example(image_encoded,image_name,target,species,yolov5_bb,detic_bb)
            writer.write(example)

In [ ]:
# create_tf_records(0, progress=True)

In [ ]:
import joblib
_ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(create_tf_records)(fold) for fold in tqdm(range(10), total=10)
    )

In [ ]:
def create_test_tf_records(fold  = 0, progress=False):
    df = test_df[test_df.split==fold]
    tfr_filename = f'/tmp/{DATASET_NAME}/happywhale-2022-test-{fold}-{df.shape[0]}.tfrec'
    with tf.io.TFRecordWriter(tfr_filename) as writer:
        it = df.iterrows()
        if progress:
            it = tqdm(it, total=len(df))
        for i,row in it:
            image_id = row.image
            target = -1
            species = -1
            image_path = f"../input/happy-whale-and-dolphin/test_images/{image_id}"
            image_encoded = tf.io.read_file(image_path)
            image_name = str.encode(image_id)
            yolov5_bb = list(read_bbox(row.yolo5_bbox))
            detic_bb = list(read_bbox(row.detic_bbox))
            example = serialize_example(image_encoded,image_name,target,species,yolov5_bb,detic_bb)
            writer.write(example)

In [ ]:
import joblib
_ = joblib.Parallel(n_jobs=8)(
        joblib.delayed(create_test_tf_records)(fold) for fold in tqdm(range(10))
    )

In [ ]:
from datetime import datetime
version_name = datetime.now().strftime("%Y%m%d-%H%M%S")
print(version_name)

In [ ]:
!ls /tmp/{DATASET_NAME}

# Verify TFRecords

In this example I load some images from TFRecords and plot with the 2 bounding boxes.

In [ ]:
from functools import partial

In [ ]:
def decode_image(image_data, bb):
    if bb is not None and bb[0] != -1:
        left, top, right, bottom = bb[0], bb[1], bb[2], bb[3]
        bbs = tf.convert_to_tensor([top, left, bottom - top, right - left])
        image = tf.io.decode_and_crop_jpeg(image_data, bbs, channels=3)
    else:
        image = tf.image.decode_jpeg(image_data, channels = 3)

    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.image.resize(image,IMAGE_SIZE_)
    return image

def read_labeled_tfrecord(example, crop_method):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        'target': tf.io.FixedLenFeature([], tf.int64),
        'species': tf.io.FixedLenFeature([], tf.int64),
        'yolov5_box': tf.io.FixedLenFeature([4], tf.int64),
        'detic_box': tf.io.FixedLenFeature([4], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    
    bb = None
    if crop_method == 'detic':
        bb = tf.cast(example['detic_box'], tf.int32)
    elif crop_method == 'yolov5':
        bb = tf.cast(example['yolov5_box'], tf.int32)

    image = decode_image(example['image'], bb)
    label = example['target']
    return image, label, example['species']

def load_dataset(filenames, crop_method, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_labeled_tfrecord, crop_method=crop_method))
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset(crop_method):
    dataset = load_dataset(TRAINING_FILENAMES, crop_method, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
# numpy and matplotlib defaults
np.set_printoptions(threshold=15, linewidth=80)
CLASSES = [0,1]

from matplotlib import patches

def batch_to_numpy_images_and_labels(data):
    images, labels, species = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    species_labels = species.numpy()
    #if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
    #    numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels, species_labels

def display_single_sample(image, label, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')

    plt.imshow(image)

    title = str(label)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch):
    """
    Display single batch Of images 
    """
    # data
    images, labels, species = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(
        images[:rows*cols],
        labels[:rows*cols]
    )):
        correct = True
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_single_sample(image, label, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32

In [ ]:
IMAGE_SIZE_ = [IMAGE_SIZE,IMAGE_SIZE]
AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob(f'/tmp/{DATASET_NAME}/happywhale-2022-train*.tfrec')
print(len(TRAINING_FILENAMES))
dataset = load_dataset(TRAINING_FILENAMES, crop_method='detic', labeled=True)
dataset = dataset.repeat()
dataset = dataset.shuffle(2048)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(AUTO) #This dataset can directly be passed to keras.fit method
print(count_data_items(TRAINING_FILENAMES))

# Displaying single batch of TFRecord
train_batch = iter(dataset)
display_batch_of_images(next(train_batch))

In [ ]:
IMAGE_SIZE_ = [IMAGE_SIZE,IMAGE_SIZE]
AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob(f'/tmp/{DATASET_NAME}/happywhale-2022-train*.tfrec')
print(len(TRAINING_FILENAMES))
dataset = load_dataset(TRAINING_FILENAMES, crop_method='yolov5', labeled=True)
dataset = dataset.repeat()
dataset = dataset.shuffle(2048)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(AUTO) #This dataset can directly be passed to keras.fit method
print(count_data_items(TRAINING_FILENAMES))

# Displaying single batch of TFRecord
train_batch = iter(dataset)
display_batch_of_images(next(train_batch))

In [ ]:
IMAGE_SIZE_ = [IMAGE_SIZE,IMAGE_SIZE]
AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob(f'/tmp/{DATASET_NAME}/happywhale-2022-test*.tfrec')
print(len(TRAINING_FILENAMES))
dataset = load_dataset(TRAINING_FILENAMES, crop_method='detic', labeled=True)
dataset = dataset.repeat()
dataset = dataset.shuffle(2048)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(AUTO) #This dataset can directly be passed to keras.fit method
print(count_data_items(TRAINING_FILENAMES))

# Displaying single batch of TFRecord
train_batch = iter(dataset)
display_batch_of_images(next(train_batch))

# Upload Dataset

In [ ]:
# !kaggle datasets version -m {version_name} -p /tmp/{DATASET_NAME} -r zip -q